In [1]:
#! pip install census

In [2]:
#dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from api_keys import census_key

In [3]:
#run Census Search to retrieve data on specific county, state, and other parameters (2018 ACS5 Census)
c = Census(census_key, year=2018)
LA_census_data = c.acs5.get(("NAME", "B01003_001E", "B01003_001E", "B01002_001E", "B19301_001E",
                             "B17001_002E"), {"for": "county:037", "in": "state:06"})
SF_census_data = c.acs5.get(("NAME", "B01003_001E", "B01003_001E", "B01002_001E", "B19301_001E",
                             "B17001_002E"), {"for": "county:075", "in": "state:06"})

#convert to dataframe & rename/reorder columns
LA_census_df = pd.DataFrame(LA_census_data)
LA_census_df = LA_census_df.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name"})
SF_census_df = pd.DataFrame(SF_census_data)
SF_census_df = SF_census_df.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name"})

#add in Poverty Rate column (Poverty Count / Population)
LA_census_df["Poverty Rate"] = 100 * \
    LA_census_df["Poverty Count"].astype(
        int) / LA_census_df["Population"].astype(int)
SF_census_df["Poverty Rate"] = 100 * \
    SF_census_df["Poverty Count"].astype(
        int) / SF_census_df["Population"].astype(int)


#combine into new dataframe
counties_census_df = LA_census_df.append(SF_census_df)
counties_census_df

,Name,Population,Median Age,Per Capita Income,Poverty Count,state,county,Poverty Rate
0,"Los Angeles County, California",10098052.0,36.2,32469.0,1589956.0,06,037,15.745175
0,"San Francisco County, California",870044.0,38.2,64157.0,92979.0,06,075,10.686701


In [4]:
#read in csv file "Count_size" for total area numbers
county_size = pd.read_csv("Resources/County_size.csv")
#place elements from Total Area column into new series
total_area = county_size["Total Area"]

#reset index on counties_census_df so the index and series match
counties_census_df = counties_census_df.reset_index(drop=True)

#add new series to a new column in orginal df
counties_census_df["Total Area (square mi.)"] = total_area

#calculate population density and place in new column
counties_census_df["Population Density"] = counties_census_df["Population"] / counties_census_df["Total Area (square mi.)"]
counties_census_df

,Name,Population,Median Age,Per Capita Income,Poverty Count,state,county,Poverty Rate,Total Area (square mi.),Population Density
0,"Los Angeles County, California",10098052.0,36.2,32469.0,1589956.0,06,037,15.745175,4751.122,2125.403641
1,"San Francisco County, California",870044.0,38.2,64157.0,92979.0,06,075,10.686701,231.890,3751.968606


In [5]:
#reorder columns
counties_census_df = counties_census_df[["Name", "Population", "Population Density",
                                         "Median Age", "Per Capita Income", "Poverty Count",
                                         "Poverty Rate", "Total Area (square mi.)"]]
counties_census_df

,Name,Population,Population Density,Median Age,Per Capita Income,Poverty Count,Poverty Rate,Total Area (square mi.)
0,"Los Angeles County, California",10098052.0,2125.403641,36.2,32469.0,1589956.0,15.745175,4751.122
1,"San Francisco County, California",870044.0,3751.968606,38.2,64157.0,92979.0,10.686701,231.890


In [6]:
#export as csv for use elsewhere
counties_census_df.to_csv("Resources/LASF_Census2018.csv", index=False, header=True)